## Introduction to global ocean biogeochemical modelling with transport operators
To model marine biogeochemical processes on a global scale we need to be able to account for the movement of chemical constituents both horizontally and vertically. We do this with a <b><i>tracer transport operator</i></b>.  When this operator acts on a tracer field it produces the advective-diffusive divergence of the tracer.

In order to represent the transport operator on a computer we have to discretize the tracer concentration field and the operator.  Once discretized the tracer field is represented as a vector and the operator is represented as a sparse matrix. (A sparse matrix behaves the same way as a regular matrix. The only difference is that in a sparse matrix the majority of the entries are zeros. These zeros are not stored explicitly to save computer memory making it possible to deal with fairly high resolution ocean models. 

Mathematically, the discretization converts an expression with partial derivatives into a matrix vector product:
$$\nabla \cdot \left[\mathbf{u}-\mathbf{K}\cdot\nabla \right]C \longrightarrow \mathbf{T}\mathbf{c}$$ where $\mathbf{T}$ is the flux divergence transport matrix and $\mathbf{c}$ is the tracer concentration vector. 

One can go a long way towards understanding what a tracer transport operator is by playing with a simple box model. We therefore introuce a simple box model before moving on to the <i>Ocean Circulation Inverse Model</i> (OCIM).


The simple box model we consider is embeded in a $2\times2\times2$ "shoebox". It has 5 <i>wet</i> boxes and 3 <i>dry</i> boxes.
<img src="boxmodel.png" width =800>

In [2]:
a = 6367e3;   # Earth radius           (m)
A = 4*pi*a^2; # Earth surface area     (m²)
d = 3700;     # ocean depth            (m)
V = 0.75*A*d; # volume of ocean        (m³)
h = 600;      # thickness of top layer (m)
#
dz = [h*ones(4,1);(d-h)*ones(4,1)]; # grid box thicknesses       (m)
dV = (dz/d).*((V/4)*ones(8,1));     # grid box volumes           (m³)
dAz = dV./dz                        # area of face ⟂ to z axis   (m²)
dy = sqrt.(dAz);                    # north-south side length    (m)
dx = sqrt.(dAz);                    # east-west side length      (m)
dAx = dV./dy;                       # area of face ⟂ to x axis   (m²)
dAy = dV./dx;                       # area of face ⟂ to y axis   (m²)
#
msk = [1, 1, 1, 0, 1, 1, 0, 0];     # wet-dry mask wet=1 dry = 0 
iwet = findall(x->x==1,msk);        # index to wet gridboxes
idry = findall(x->x==0,msk);        # index to dry gridboxes

The circulation consists of 
<ul>
    <li>a meridional overturning circulation flowing from box 1 to box 2 to box 6 to box 5 and back to box 1</li>
    <li>a zonal current in a reentrant channel from box 1 to box 3 and back to box 1</li>
    <li>vertical mixing representing deep convection between box 2 and box 6</li>
</ul>
    

In [13]:
using SparseArrays
TRdiv = spzeros(8,8);
# "Antarctic Circumpoloar Current"
acc = 100e6;  # (m³/s)
TRdiv += sparse([1,1],[1,3],dV[1]\[-acc,acc],8,8);
TRdiv += sparse([3,3],[3,1],dV[3]\[-acc,acc],8,8);
# "Meridional Overturning Circulation"
moc = 15e6    # (m³/s)
TRdiv += sparse([1,1],[1,5],dV[1]\[-moc,moc],8,8);
TRdiv += sparse([2,2],[2,1],dV[2]\[-moc,moc],8,8);
TRdiv += sparse([5,5],[5,6],dV[5]\[-moc,moc],8,8);
TRdiv += sparse([6,6],[6,2],dV[6]\[-moc,moc],8,8);
# vertical mixing at "high northern latitudes"
q = 10e6      # (m³/s)
TRdiv += sparse([2,2],[2,6],dV[2]\[-q,q],8,8);
TRdiv += sparse([6,6],[6,2],dV[6]\[-q,q],8,8);